In [1]:
import os
import json
import math
import rasterio
from rasterio.plot import reshape_as_image
import rasterio.mask
from rasterio.features import rasterize
import matplotlib
import matplotlib.pyplot as plt
import cv2
import pandas as pd
import geopandas as gpd
from shapely.geometry import mapping, Point, Polygon
from shapely.ops import cascaded_union


import matplotlib.pyplot as plt


### Open image with rasterio:

In [2]:
with rasterio.open("LFMP_ortho.tifa") as src:
    img = src.read()
    meta = src.meta

img = reshape_as_image(img)

plt.imshow(img)

RasterioIOError: LFMP_ortho.tifa: No such file or directory

### Read Geojson as  a dataframe:

In [205]:
train_df = gpd.read_file('product_ADBLucem_LFMP_1905_2.json')


train_df = train_df.drop(columns=['originated', 'readonly',
       'notvalidated', 'lock', 'elev', 'hacc', 'iata', 'name', 'idarpt',
       'acft', 'idnumber', 'termref', 'pcn', 'restacft', 'status', 'surftype',
       'length', 'width', 'color', 'direc', 'style', 'rwymktyp', 'asda',
       'availPavedSurfFromThr', 'brngmag', 'brngtrue', 'cat', 'ellipse',
       'geound', 'lda', 'rops_landing_length', 'rwyslope', 'tdze', 'tdzslope',
       'thrtype', 'toda', 'tora', 'vasis', 'bridge', 'gsurftyp', 'runwayexit',
       'imagery_date'])

#### Only polygon:

In [206]:
train_df = train_df[train_df["geometry"].geom_type == 'Polygon']

In [207]:
features_list = []
for e in train_df['feattype']:
    if e['value'] not in features_list:
        features_list.append(e['value'])

print(features_list)

['Apron.apronelement', 'FinalApproachAndTakeOffArea.finalapproachandtakeoffarea', 'ParkingStandArea.parkingstandarea', 'Runway.runwaydisplacedarea', 'Runway.runwayelement', 'Runway.runwaymarking', 'Runway.runwayshoulder', 'ServiceRoad.serviceroad', 'Stopway.stopway', '0', 'Taxiway.taxiwayshoulder', 'TouchDownLiftOffArea.touchdownliftoffarea', 'VerticalStructure.verticalpolygonalstructure', 'Water.water']


In [208]:
outfolder = "/Users/salimbaddou/PIE"  #folder where the mask is stored
os.makedirs(outfolder, exist_ok=True)

In [211]:
def poly_from_utm(polygon, transform):
    poly_pts = []
    
    # make a polygon from multipolygon
    poly = cascaded_union(polygon)
    for i in np.array(poly.exterior.coords):
        
        # transfrom polygon to image crs, using raster meta
        poly_pts.append(~transform * tuple(i))
        
    # make a shapely Polygon object
    new_poly = Polygon(poly_pts)
    return new_poly

# creating binary mask for field/not_filed segmentation.


im_size = (src.meta['height'], src.meta['width'])
whole_mask = np.zeros([src.meta['height'],src.meta['width']])

for i,f in enumerate(features_list):
    poly_shp = []
    data_geom = train_df[train_df['feattype'] == {'value': f}].geometry
    for g in data_geom.values:


        poly = poly_from_utm(g, src.meta['transform'])
        poly_shp.append(poly)

        if g.geom_type == 'Polygon':
            poly = poly_from_utm(g, src.meta['transform'])
            poly_shp.append(poly)
        else:
            for p in g:
                poly = poly_from_utm(p, src.meta['transform'])
                poly_shp.append(poly)


    mask = rasterize(shapes=poly_shp,out_shape=im_size)*math.floor(255/len(features_list))*(i+1)
    whole_mask = whole_mask + mask
        



In [3]:
plt.figure(figsize=(15,15))
plt.imshow(whole_mask)

NameError: name 'whole_mask' is not defined

<Figure size 1080x1080 with 0 Axes>

In [214]:
matplotlib.image.imsave('name.png', whole_mask) #save img in out_folder